In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pickle
import gzip

In [2]:
import pandas as pd
import sklearn

print(f'pandas version: {pd.__version__}')
print(f'scikit-learn version: {sklearn.__version__}')


pandas version: 2.2.2
scikit-learn version: 1.4.2


In [3]:
# Load the dataset
data_path = 'data_new.csv'
df = pd.read_csv(data_path)


In [4]:
# Handle age bands to create a consolidated 'age' feature
age_band_columns = [
    'dem_age_band_18-24_tm1', 'dem_age_band_25-34_tm1', 'dem_age_band_35-44_tm1',
    'dem_age_band_45-54_tm1', 'dem_age_band_55-64_tm1', 'dem_age_band_65-74_tm1', 'dem_age_band_75+_tm1'
]

# Check if age band columns exist
if any(col in df.columns for col in age_band_columns):
    # Create a numerical 'age' feature based on age bands
    age_mapping = {
        'dem_age_band_18-24_tm1': 21,
        'dem_age_band_25-34_tm1': 30,
        'dem_age_band_35-44_tm1': 40,
        'dem_age_band_45-54_tm1': 50,
        'dem_age_band_55-64_tm1': 60,
        'dem_age_band_65-74_tm1': 70,
        'dem_age_band_75+_tm1': 80
    }
    df['age'] = sum(df[band] * age_mapping[band] for band in age_band_columns if band in df.columns)
else:
    print("No age bands found in the dataset.")

# Handle missing values by filling with median for numeric columns
df.fillna(df.median(numeric_only=True), inplace=True)

# Encode the categorical variable 'race' if it is non-numeric
if 'race' in df.columns and df['race'].dtype == 'object':
    df['race'] = LabelEncoder().fit_transform(df['race'])

# Consolidate "biomarkers" into a single column
biomarker_columns = [
    'cre_min-low_tm1', 'cre_min-high_tm1', 'cre_min-normal_tm1',
    'cre_mean-low_tm1', 'cre_mean-high_tm1', 'cre_mean-normal_tm1',
    'cre_max-low_tm1', 'cre_max-high_tm1', 'cre_max-normal_tm1',
    'crp_min-low_tm1', 'crp_min-high_tm1', 'crp_min-normal_tm1',
    'crp_mean-low_tm1', 'crp_mean-high_tm1', 'crp_mean-normal_tm1',
    'crp_max-low_tm1', 'crp_max-high_tm1', 'crp_max-normal_tm1',
    'esr_min-low_tm1', 'esr_min-high_tm1', 'esr_min-normal_tm1',
    'esr_mean-low_tm1', 'esr_mean-high_tm1', 'esr_mean-normal_tm1',
    'esr_max-low_tm1', 'esr_max-high_tm1', 'esr_max-normal_tm1',
    'ghba1c_min-low_tm1', 'ghba1c_min-high_tm1', 'ghba1c_min-normal_tm1',
    'ghba1c_mean-low_tm1', 'ghba1c_mean-high_tm1', 'ghba1c_mean-normal_tm1',
    'ghba1c_max-low_tm1', 'ghba1c_max-high_tm1', 'ghba1c_max-normal_tm1',
    'hct_min-low_tm1', 'hct_min-high_tm1', 'hct_min-normal_tm1',
    'hct_mean-low_tm1', 'hct_mean-high_tm1', 'hct_mean-normal_tm1',
    'hct_max-low_tm1', 'hct_max-high_tm1', 'hct_max-normal_tm1',
    'ldl_min-low_tm1', 'ldl_min-high_tm1', 'ldl_min-normal_tm1',
    'ldl-mean-low_tm1', 'ldl-mean-high_tm1', 'ldl-mean-normal_tm1',
    'ldl_max-low_tm1', 'ldl_max-high_tm1', 'ldl_max-normal_tm1',
    'nt_bnp_min-low_tm1', 'nt_bnp_min-high_tm1', 'nt_bnp_min-normal_tm1',
    'nt_bnp_mean-low_tm1', 'nt_bnp_mean-high_tm1', 'nt_bnp_mean-normal_tm1',
    'nt_bnp_max-low_tm1', 'nt_bnp_max-high_tm1', 'nt_bnp_max-normal_tm1',
    'sodium_min-low_tm1', 'sodium_min-high_tm1', 'sodium_min-normal_tm1',
    'sodium_mean-low_tm1', 'sodium_mean-high_tm1', 'sodium_mean-normal_tm1',
    'sodium_max-low_tm1', 'sodium_max-high_tm1', 'sodium_max-normal_tm1',
    'trig_min-low_tm1', 'trig_min-high_tm1', 'trig_min-normal_tm1',
    'trig_mean-low_tm1', 'trig_mean-high_tm1', 'trig_mean-normal_tm1',
    'trig_max-low_tm1', 'trig_max-high_tm1', 'trig_max-normal_tm1'
]

# Create the "biomarkers" feature by checking if any values indicate "normal"
if all(col in df.columns for col in biomarker_columns):
    df['biomarkers'] = df[biomarker_columns].apply(lambda row: 1 if any('normal' in col and row[col] > 0 for col in biomarker_columns) else 0, axis=1)
else:
    print("Some biomarker columns are missing from the dataset.")

# Consolidate "comorbidity" into a single column
comorbidity_columns = [
    'alcohol_elixhauser_tm1', 'anemia_elixhauser_tm1', 'arrhythmia_elixhauser_tm1',
    'arthritis_elixhauser_tm1', 'bloodlossanemia_elixhauser_tm1', 'coagulopathy_elixhauser_tm1',
    'compdiabetes_elixhauser_tm1', 'depression_elixhauser_tm1', 'drugabuse_elixhauser_tm1',
    'electrolytes_elixhauser_tm1', 'hypertension_elixhauser_tm1', 'hypothyroid_elixhauser_tm1',
    'liver_elixhauser_tm1', 'neurodegen_elixhauser_tm1', 'obesity_elixhauser_tm1',
    'paralysis_elixhauser_tm1', 'psychosis_elixhauser_tm1', 'pulmcirc_elixhauser_tm1',
    'pvd_elixhauser_tm1', 'renal_elixhauser_tm1', 'uncompdiabetes_elixhauser_tm1',
    'valvulardz_elixhauser_tm1', 'wtloss_elixhauser_tm1', 'cerebrovasculardz_romano_tm1',
    'chf_romano_tm1', 'dementia_romano_tm1', 'hemiplegia_romano_tm1', 'hivaids_romano_tm1',
    'metastatic_romano_tm1', 'myocardialinfarct_romano_tm1', 'pulmonarydz_romano_tm1',
    'tumor_romano_tm1', 'ulcer_romano_tm1'
]

# Create the "comorbidity" feature by summing the relevant columns if columns are available
if all(col in df.columns for col in comorbidity_columns):
    df['comorbidity'] = df[comorbidity_columns].sum(axis=1)
else:
    print("Some comorbidity columns are missing from the dataset.")


In [5]:

# Define the features to be used for model training
features = ['cost_t', 'age', 'dem_female', 'race', 'biomarkers', 'comorbidity']

# Check that all features are available in the dataset
missing_features = [feature for feature in features if feature not in df.columns]
if missing_features:
    print(f"Missing features from the dataset: {missing_features}")


In [6]:

# Proceed only if no features are missing
if not missing_features:
    # Prepare the features (X) and target (y)
    X = df[features]
    y = df['risk_score_t']

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Standardize the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)


# Save the scaler in binary
    with open('standard_scaler.pkl', 'wb') as scaler_file_out:
        pickle.dump(scaler, scaler_file_out)

# Save the scaler in binary
    with open('standard_scaler.pkl', 'rb') as scaler_file_in:
        with gzip.open('standard_scaler.pkl.gz', 'wb') as scaler_file_out:
            scaler_file_out.write(scaler_file_in.read())  # Properly write the binary contents to gzip


    # Train the Random Forest model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f'Mean Squared Error: {mse}')
    print(f'R-squared: {r2}')

    # Feature importance
    feature_importance = model.feature_importances_
    feature_importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importance})
    print(feature_importance_df)
    # Save the random forest model in binary
    with open('random_forest_risk_score_model.pkl', 'wb') as model_file_out:
        pickle.dump(model, model_file_out)
    # Save the random forest model in binary
    with open('random_forest_risk_score_model.pkl', 'rb') as model_file_in:
        with gzip.open('random_forest_risk_score_model_compressed.pkl.gz', 'wb') as model_file_out:
            model_file_out.write(model_file_in.read())  # Properly write the binary contents to gzip
else:
    print("Cannot train the model due to missing features.")

Mean Squared Error: 15.702192304246264
R-squared: 0.4454525483955426
       Feature  Importance
0       cost_t    0.412919
1          age    0.088728
2   dem_female    0.030780
3         race    0.024531
4   biomarkers    0.024343
5  comorbidity    0.418699


In [7]:
# 1. White, Good Health
white_good_health = pd.DataFrame({
    'cost_t': [1000],      # Low cost
    'age': [30],           # Younger age
    'dem_female': [0],     # Male
    'race': [0],           # White
    'biomarkers': [1],     # Normal biomarkers
    'comorbidity': [1]     # Low comorbidity
})

# 2. Black, Good Health
black_good_health = pd.DataFrame({
    'cost_t': [1200],      # Low cost
    'age': [35],           # Younger age
    'dem_female': [1],     # Female
    'race': [1],           # Black
    'biomarkers': [1],     # Normal biomarkers
    'comorbidity': [2]     # Low comorbidity
})

# 3. White, Bad Health
white_bad_health = pd.DataFrame({
    'cost_t': [4000],      # High cost
    'age': [60],           # Older age
    'dem_female': [1],     # Female
    'race': [0],           # White
    'biomarkers': [0],     # Abnormal biomarkers
    'comorbidity': [4]     # High comorbidity
})

# 4. Black, Bad Health
black_bad_health = pd.DataFrame({
    'cost_t': [4500],      # High cost
    'age': [55],           # Older age
    'dem_female': [0],     # Male
    'race': [1],           # Black
    'biomarkers': [0],     # Abnormal biomarkers
    'comorbidity': [3]     # High comorbidity
})



# Decompress and load the saved random forest model
with gzip.open('random_forest_risk_score_model_compressed.pkl.gz', 'rb') as model_file_in:
    model = pickle.load(model_file_in)

# Decompress and load the saved scaler
with gzip.open('standard_scaler.pkl.gz', 'rb') as scaler_file_in:
    scaler = pickle.load(scaler_file_in)

# Define the features used for prediction
features = ['cost_t', 'age', 'dem_female', 'race', 'biomarkers', 'comorbidity']

# Make prediction for each dummy data entry separately

# 1. White, Good Health
if all(feature in white_good_health.columns for feature in features):
    X_white_good_health = white_good_health[features]
    X_white_good_health = scaler.transform(X_white_good_health)
    predicted_risk_score_white_good_health = model.predict(X_white_good_health)[0]
    print(f"White, Good Health: Predicted Risk Score = {predicted_risk_score_white_good_health}")
else:
    print("White, Good Health: Missing required features")

# 2. Black, Good Health
if all(feature in black_good_health.columns for feature in features):
    X_black_good_health = black_good_health[features]
    X_black_good_health = scaler.transform(X_black_good_health)
    predicted_risk_score_black_good_health = model.predict(X_black_good_health)[0]
    print(f"Black, Good Health: Predicted Risk Score = {predicted_risk_score_black_good_health}")
else:
    print("Black, Good Health: Missing required features")

# 3. White, Bad Health
if all(feature in white_bad_health.columns for feature in features):
    X_white_bad_health = white_bad_health[features]
    X_white_bad_health = scaler.transform(X_white_bad_health)
    predicted_risk_score_white_bad_health = model.predict(X_white_bad_health)[0]
    print(f"White, Bad Health: Predicted Risk Score = {predicted_risk_score_white_bad_health}")
else:
    print("White, Bad Health: Missing required features")

# 4. Black, Bad Health
if all(feature in black_bad_health.columns for feature in features):
    X_black_bad_health = black_bad_health[features]
    X_black_bad_health = scaler.transform(X_black_bad_health)
    predicted_risk_score_black_bad_health = model.predict(X_black_bad_health)[0]
    print(f"Black, Bad Health: Predicted Risk Score = {predicted_risk_score_black_bad_health}")
else:
    print("Black, Bad Health: Missing required features")


White, Good Health: Predicted Risk Score = 1.8827092787174313
Black, Good Health: Predicted Risk Score = 4.359724952208392
White, Bad Health: Predicted Risk Score = 6.941566162731443
Black, Bad Health: Predicted Risk Score = 6.116463232141847


In [8]:
# Assuming 'df' is your DataFrame and 'risk_score_t' is the column with the risk scores
min_risk_score = df['risk_score_t'].min()
max_risk_score = df['risk_score_t'].max()

print(f"Minimum Risk Score: {min_risk_score}")
print(f"Maximum Risk Score: {max_risk_score}")

# Statistical summary of the 'risk_score_t' column
risk_score_description = df['risk_score_t'].describe()
print(risk_score_description)


Minimum Risk Score: 0.0
Maximum Risk Score: 100.0
count    48784.000000
mean         4.393692
std          5.519582
min          0.000000
25%          1.443859
50%          2.887719
75%          5.350773
max        100.000000
Name: risk_score_t, dtype: float64
